In [2]:
import pandas as pd
import numpy as np
import sys, os,re,codecs,csv
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers import GlobalMaxPool1D
from keras.models import Model
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import optimizers
from keras import layers


In [23]:
path = 'D:/'
comp = 'kaggle_toxic_comment/jigsaw-toxic-comment-classification-challenge/'

EMBEDDING_FILE=f'D:/kaggle_fake_news/glove.6B/glove.6B.50d.txt'
TRAIN_DATA_FILE=f'D:/kaggle_toxic_comment/jigsaw-toxic-comment-classification-challenge/train/train.csv'
TEST_DATA_FILE=f'D:/kaggle_toxic_comment/jigsaw-toxic-comment-classification-challenge/test/test.csv'

In [24]:
embed_size = 50     # how big is each word vector
max_features = 20000   # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100    #max number of words in a comment to use

In [25]:
TRAIN_DATA_FILE

'D:/kaggle_toxic_comment/jigsaw-toxic-comment-classification-challenge/train/train.csv'

In [26]:
train = pd.read_csv('D:/kaggle_toxic_comment/jigsaw-toxic-comment-classification-challenge/train/train.csv')
test = pd.read_csv('D:/kaggle_toxic_comment/jigsaw-toxic-comment-classification-challenge/test/test.csv')

In [27]:
train['comment_text']

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [28]:
test['comment_text']

0         Yo bitch Ja Rule is more succesful then you'll...
1         == From RfC == \n\n The title is fine as it is...
2         " \n\n == Sources == \n\n * Zawe Ashton on Lap...
3         :If you have a look back at the source, the in...
4                 I don't anonymously edit articles at all.
                                ...                        
153159    . \n i totally agree, this stuff is nothing bu...
153160    == Throw from out field to home plate. == \n\n...
153161    " \n\n == Okinotorishima categories == \n\n I ...
153162    " \n\n == ""One of the founding nations of the...
153163    " \n :::Stop already. Your bullshit is not wel...
Name: comment_text, Length: 153164, dtype: object

In [29]:
#list_sentences_train
train_sentences = train['comment_text']
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
test_sentences = test['comment_text']

### Standard keras preprocessing, to turn each comment into a list of word indexes of equal length (with truncation or padding as needed).

In [30]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_sentences))


list_tokenizer_train = tokenizer.texts_to_sequences(train_sentences)
list_tokenizer_test = tokenizer.texts_to_sequences(test_sentences)


X_t  = pad_sequences(list_tokenizer_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenizer_test, maxlen=maxlen)

In [31]:
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype = 'float32')

embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE,encoding="utf8"))

In [32]:
#Use these vectors to create our embedding matrix, with random initialization for words that aren't in GloVe. We'll use the same mean and stdev of embeddings the GloVe has when generating the random init.


In [33]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.020940498, 0.6441043)

In [34]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [35]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1);

Epoch 1/2
4488/4488 [==============================] - 3509s 782ms/step - loss: 0.0595 - accuracy: 0.9417 - val_loss: 0.0499 - val_accuracy: 0.9940
Epoch 2/2
4488/4488 [==============================] - 4341s 967ms/step - loss: 0.0444 - accuracy: 0.9774 - val_loss: 0.0457 - val_accuracy: 0.9940
